In [1]:
import numpy as np, pandas as pd

# cell key perturbation example

checking out example from explanation at https://gss.civilservice.gov.uk/wp-content/uploads/2017/01/ExN-Disclosure-control-methodology-in-2021-Census-outputs-Spicer-Blanchard-Dove-ONS.docx

In [2]:
n = 1_000 # number of records
b = 200 # parameter related to spread/granularity of noise

In [3]:
M = 200 # max count of any of the binned cells (this is a guess/overshoot)

In [4]:
# this is a table of values that from which we obtain the 'random noise' with which to perturb counts
Ptable = np.random.randint(-1,2, size = [M,b]) #just adding [-1,1]; could tweak this as desired

In [ ]:
#question: would census want noise to be some function of count size? what would the distribution/range be?

In [5]:
record_key = pd.DataFrame({'sex_id':np.random.choice(2,n), #assign random sexes
                           'age':np.random.randint(0, 116, size = n), # assign random ages
                           'key':np.random.choice(a = n, size = n, replace=False), #create unique random key
                           'pweight': 1})

In [ ]:
#question: is this key assignment sufficiently random? what would happen if it was from some distribution w/out replacement?

In [6]:
record_key['5yr_age'] = [i - (i % 5) for i in record_key.age] #bin the data

In [7]:
record_key.head() #this is the underlying / 'true' data

,sex_id,age,key,pweight,5yr_age
0,1,32,517,1,30
1,1,44,199,1,40
2,1,23,997,1,20
3,1,54,348,1,50
4,1,56,526,1,55


In [8]:
freq_table = record_key[['sex_id','5yr_age','key','pweight']]
freq_table = freq_table.groupby(['sex_id','5yr_age']).sum().reset_index() #bin by sex and 5-year ages

In [9]:
freq_table['cell_key'] = [(i % b) for i in freq_table.key] #cell key is ((sum of random keys in bin) mod b)
freq_table.head()

,sex_id,5yr_age,key,pweight,cell_key
0,0,0,9397,17,197
1,0,5,9872,20,72
2,0,10,10939,20,139
3,0,15,11858,26,58
4,0,20,10514,18,114


In [10]:
freq_table.pweight.max() # M needed to be geq this

35

In [11]:
def add_noise(df, Ptable):
    df['noise'] = [Ptable[i -1][j] for (i,j) in zip(df.pweight, df.cell_key)]
    df['noisy_pweight'] = df.pweight + df.noise
    
    return df

In [12]:
output = add_noise(freq_table, Ptable)

In [13]:
output.head()

,sex_id,5yr_age,key,pweight,cell_key,noise,noisy_pweight
0,0,0,9397,17,197,1,18
1,0,5,9872,20,72,1,21
2,0,10,10939,20,139,-1,19
3,0,15,11858,26,58,1,27
4,0,20,10514,18,114,-1,17


In [14]:
output.noise.value_counts() #distribution of noise added

 0    20
 1    15
-1    13
Name: noise, dtype: int64